# Path Loss Prediction with Supervised Learning
## EE 48009 & EE 58009 - Assignment 1

This notebook implements ray-traced wireless propagation data generation using Sionna and applies supervised learning to predict path loss.

**Key Concepts:**
- **Ray Tracing**: Computational technique to model electromagnetic wave propagation by tracing paths from transmitter to receiver
- **Path Loss**: Reduction in power density of an electromagnetic wave as it propagates through space
- **Supervised Learning**: Training models to predict path loss (target) from physical/environmental features (inputs)

**Bridge to Wireless Communications:**
- We use **regression models** (ML concept) for **channel propagation modeling** (wireless concept)
- Features like AoA/AoD, delay spread → inputs to **neural networks** or **random forests**
- Goal: Replace expensive ray tracing with fast ML inference for **network planning** and **resource allocation**

## Step 0: Installation and Imports

In [ ]:
# Install Sionna (uncomment if running for first time)
# !pip install sionna

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow warnings

# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# TensorFlow and Sionna
import tensorflow as tf
import sionna
from sionna.rt import load_scene, PlanarArray, Transmitter, Receiver, Camera

# ML libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.inspection import permutation_importance

# Deep Learning
from tensorflow import keras
from tensorflow.keras import layers, models

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"Sionna version: {sionna.__version__}")
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

## Step 1: Choose the Scenario

**Scene Selection Rationale:**
- We'll use the **Munich** scene (urban outdoor) or **Etoile** scene (urban intersection)
- These scenes are appropriate for 28 GHz mmWave study because:
  - Dense multipath environment with buildings causing reflections/diffractions
  - Realistic for 5G urban deployment scenarios
  - Sufficient complexity to demonstrate ML model learning

**Wireless Context:**
- 28 GHz is in the **mmWave band** (24-100 GHz)
- High path loss and blockage → need for accurate propagation models
- Critical for 5G NR FR2 (Frequency Range 2) network planning

In [ ]:
# Load scene - choose one of the following:
# Options: 'etoile', 'munich', 'simple_street_canyon', 'simple_wedge'

SCENE_NAME = 'munich'  # Change this to experiment with different scenes
CARRIER_FREQUENCY = 28e9  # 28 GHz (mmWave band)

# Load the scene
scene = load_scene(SCENE_NAME)
scene.frequency = CARRIER_FREQUENCY
scene.synthetic_array = True  # Use synthetic arrays for faster computation

print(f"Scene loaded: {SCENE_NAME}")
print(f"Carrier frequency: {CARRIER_FREQUENCY/1e9} GHz")

# Visualize the scene (if running locally with display)
# scene.preview()

## Step 2: Define the Transmitter (TX)

**Antenna Array Concept:**
- **Uniform Planar Array (UPA)**: M×M grid of antenna elements
- Increasing M → higher **beamforming gain** and **spatial selectivity**
- In ML terms: M is a **hyperparameter** that affects the input-output relationship

**Bridge to Communications:**
- Larger arrays → narrower beams → better **SNR** at receiver
- We'll compare M=1,4,8 to see how array size affects path loss characteristics
- This is analogous to **feature engineering** in ML: changing M changes the feature space

In [ ]:
def setup_transmitter(scene, M=4, position=None, orientation=None):
    """
    Configure transmitter with M×M UPA
    
    Parameters:
    -----------
    scene : Sionna scene object
    M : int
        Antenna array dimension (M×M array)
    position : list or None
        [x, y, z] coordinates in meters
    orientation : list or None
        [yaw, pitch, roll] in degrees
    
    Returns:
    --------
    tx : Transmitter object
    """
    # Default positions for different scenes (you may need to adjust these)
    default_positions = {
        'munich': [50.0, 0.0, 25.0],  # x, y, z (height = 25m)
        'etoile': [0.0, 0.0, 30.0],
        'simple_street_canyon': [0.0, 0.0, 20.0]
    }
    
    if position is None:
        position = default_positions.get(SCENE_NAME, [0.0, 0.0, 25.0])
    
    if orientation is None:
        orientation = [0.0, -10.0, 0.0]  # Slight downward tilt
    
    # Create antenna array
    # num_rows, num_cols for UPA
    antenna_array = PlanarArray(num_rows=M, 
                                num_cols=M,
                                vertical_spacing=0.5,  # in wavelengths
                                horizontal_spacing=0.5,
                                pattern="iso",  # isotropic pattern
                                polarization="V")  # vertical polarization
    
    # Apply rotation if needed
    # antenna_array.rotate(orientation)
    
    # Add transmitter to scene
    tx = Transmitter(name="tx",
                    position=position,
                    orientation=orientation)
    scene.add(tx)
    tx.array = antenna_array
    
    print(f"Transmitter configured:")
    print(f"  Position: {position}")
    print(f"  Orientation: {orientation}°")
    print(f"  Array size: {M}×{M} = {M*M} elements")
    print(f"  Array gain: ~{10*np.log10(M*M):.1f} dB")
    
    return tx

# Test with M=4 (we'll loop over M=1,4,8 later)
M_ARRAY = 4
tx = setup_transmitter(scene, M=M_ARRAY)

## Step 3: Define the Receiver (RX) Grid

**Grid Design:**
- Create uniform 2D grid of receiver locations
- Spacing determines **dataset size** (more points → more training samples)
- In ML terms: each RX point is one **training example**

**Practical Considerations:**
- Small spacing (2m) → more data → better model generalization
- Large spacing (10m) → faster computation but possible underfitting
- Trade-off between **computational cost** and **model accuracy**

In [ ]:
def create_rx_grid(scene, x_range, y_range, z_height=1.5, spacing=5.0):
    """
    Create a uniform grid of receiver locations
    
    Parameters:
    -----------
    scene : Sionna scene
    x_range : tuple
        (x_min, x_max) in meters
    y_range : tuple
        (y_min, y_max) in meters
    z_height : float
        Receiver height (e.g., 1.5m for user equipment)
    spacing : float
        Grid spacing in meters
    
    Returns:
    --------
    rx_positions : np.ndarray
        Array of shape (num_points, 3) with [x, y, z] coordinates
    """
    x_min, x_max = x_range
    y_min, y_max = y_range
    
    # Create grid
    x_coords = np.arange(x_min, x_max + spacing, spacing)
    y_coords = np.arange(y_min, y_max + spacing, spacing)
    
    # Create meshgrid
    X, Y = np.meshgrid(x_coords, y_coords)
    
    # Flatten and add z coordinate
    rx_positions = np.column_stack([
        X.ravel(),
        Y.ravel(),
        np.full(X.size, z_height)
    ])
    
    print(f"RX Grid created:")
    print(f"  X range: [{x_min}, {x_max}] m")
    print(f"  Y range: [{y_min}, {y_max}] m")
    print(f"  Z height: {z_height} m")
    print(f"  Spacing: {spacing} m")
    print(f"  Total RX points: {len(rx_positions)}")
    print(f"  → This will be our dataset size")
    
    return rx_positions

# Define grid parameters (adjust based on your scene)
# For Munich scene:
RX_GRID = create_rx_grid(scene,
                        x_range=(-50, 100),
                        y_range=(-50, 50),
                        z_height=1.5,
                        spacing=5.0)  # 5m spacing

# Visualize grid
plt.figure(figsize=(10, 6))
plt.scatter(RX_GRID[:, 0], RX_GRID[:, 1], s=1, alpha=0.5)
plt.xlabel('X coordinate (m)')
plt.ylabel('Y coordinate (m)')
plt.title('Receiver Grid Layout')
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.show()

## Step 4 & 5: Perform Ray Tracing and Compute Path Loss

**Ray Tracing Process:**
1. Shoot rays from TX to each RX position
2. Trace reflections, diffractions through environment
3. Compute complex channel gain h = Σ_ℓ a_ℓ for all paths ℓ
4. Calculate received power: P_RX = P_TX |h|²
5. Path loss: PL[dB] = P_TX[dB] - P_RX[dB]

**Channel Model Bridge:**
- Each path has complex gain a_ℓ (amplitude + phase)
- **Coherent combining**: vector sum of all path contributions
- This is the **ground truth** label for our supervised learning

**Features Extracted:**
- LoS/NLoS indicator → **binary classification** feature
- Number of reflections → affects **multipath richness**
- AoA/AoD → spatial characteristics for **beamforming**
- Delay spread → affects **ISI** in OFDM systems

In [ ]:
def perform_ray_tracing_and_extract_features(scene, rx_positions, tx_power_dbm=30.0):
    """
    Perform ray tracing for all RX positions and extract features
    
    Parameters:
    -----------
    scene : Sionna scene with configured TX
    rx_positions : np.ndarray
        RX coordinates
    tx_power_dbm : float
        Transmit power in dBm
    
    Returns:
    --------
    dataset : pd.DataFrame
        Dataset with features and path loss labels
    """
    # Convert TX power from dBm to Watts
    tx_power_w = 10**((tx_power_dbm - 30) / 10)
    
    # Storage for dataset
    data_records = []
    
    print("Starting ray tracing...")
    
    for idx, rx_pos in enumerate(tqdm(rx_positions, desc="Ray tracing")):
        # Add receiver to scene temporarily
        rx = Receiver(name=f"rx_{idx}",
                     position=rx_pos.tolist(),
                     orientation=[0, 0, 0])
        
        # Single antenna at receiver for simplicity
        rx.array = PlanarArray(num_rows=1, num_cols=1,
                              vertical_spacing=0.5,
                              horizontal_spacing=0.5,
                              pattern="iso",
                              polarization="V")
        scene.add(rx)
        
        try:
            # Compute propagation paths
            paths = scene.compute_paths(max_depth=5,  # max reflections/diffractions
                                       num_samples=int(1e6))  # ray density
            
            # Extract path information
            # paths.a: complex path gains [num_rx, num_rx_ant, num_tx, num_tx_ant, num_paths, num_time_steps]
            # paths.tau: path delays
            # paths.theta_r, phi_r: RX azimuth and elevation angles (AoA)
            # paths.theta_t, phi_t: TX azimuth and elevation angles (AoD)
            
            # Get channel gains (squeeze extra dimensions for single RX/TX)
            path_gains = paths.a[0, 0, 0, :, :, 0].numpy()  # [num_tx_ant, num_paths]
            path_delays = paths.tau[0, 0, :].numpy()  # [num_paths]
            
            # Coherent combining across all paths and antennas
            h_combined = np.sum(path_gains)  # complex channel coefficient
            
            # Received power
            rx_power_w = tx_power_w * np.abs(h_combined)**2
            rx_power_dbm = 10 * np.log10(rx_power_w) + 30
            
            # Path loss in dB
            path_loss_db = tx_power_dbm - rx_power_dbm
            
            # Extract features
            num_paths = path_gains.shape[1]
            
            # LoS indicator (first path with shortest delay is typically LoS)
            los_indicator = 1 if num_paths > 0 and np.min(path_delays) < 1e-8 else 0
            
            # Distance from TX to RX
            tx_pos = np.array(scene.transmitters['tx'].position)
            distance_3d = np.linalg.norm(rx_pos - tx_pos)
            
            # Mean and std of path delays (delay spread)
            mean_delay = np.mean(path_delays) if num_paths > 0 else 0
            std_delay = np.std(path_delays) if num_paths > 0 else 0
            
            # Angles (simplified - you may want to extract per-path angles)
            # For now, we'll use aggregate statistics
            
            # Create record
            record = {
                # Target variable
                'path_loss_db': path_loss_db,
                
                # RX position features
                'rx_x': rx_pos[0],
                'rx_y': rx_pos[1],
                'rx_z': rx_pos[2],
                
                # Distance feature
                'distance_3d': distance_3d,
                
                # LoS/NLoS
                'los_indicator': los_indicator,
                
                # Path statistics
                'num_paths': num_paths,
                'mean_delay': mean_delay,
                'delay_spread': std_delay,
                
                # Received power (for reference)
                'rx_power_dbm': rx_power_dbm,
                
                # Scene and configuration
                'carrier_freq_ghz': CARRIER_FREQUENCY / 1e9,
                'tx_array_size': M_ARRAY,
                'environment': SCENE_NAME
            }
            
            data_records.append(record)
            
        except Exception as e:
            print(f"Error at RX position {idx}: {e}")
            continue
        
        finally:
            # Remove receiver from scene
            scene.remove(f"rx_{idx}")
    
    # Create DataFrame
    dataset = pd.DataFrame(data_records)
    
    print(f"\nRay tracing complete!")
    print(f"Dataset size: {len(dataset)} samples")
    print(f"Features: {list(dataset.columns)}")
    
    return dataset

# Perform ray tracing (this may take several minutes)
dataset = perform_ray_tracing_and_extract_features(scene, RX_GRID[:100])  # Start with 100 points for testing

## Step 6: Dataset Analysis and Visualization

**Exploratory Data Analysis (EDA):**
- Check for missing values, outliers
- Visualize feature distributions
- Correlation analysis between features and target

**ML Context:**
- Understanding data distribution is crucial for model selection
- Outliers may indicate measurement errors or rare propagation conditions
- Feature correlation helps avoid **multicollinearity** in linear models

In [ ]:
# Display dataset summary
print("Dataset Statistics:")
print(dataset.describe())

print("\nMissing values:")
print(dataset.isnull().sum())

# Visualizations
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Path loss distribution
axes[0, 0].hist(dataset['path_loss_db'], bins=50, edgecolor='black')
axes[0, 0].set_xlabel('Path Loss (dB)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Path Loss Distribution')

# Path loss vs distance
axes[0, 1].scatter(dataset['distance_3d'], dataset['path_loss_db'], alpha=0.5)
axes[0, 1].set_xlabel('Distance (m)')
axes[0, 1].set_ylabel('Path Loss (dB)')
axes[0, 1].set_title('Path Loss vs Distance')

# LoS vs NLoS comparison
dataset.boxplot(column='path_loss_db', by='los_indicator', ax=axes[0, 2])
axes[0, 2].set_xlabel('LoS Indicator (0=NLoS, 1=LoS)')
axes[0, 2].set_ylabel('Path Loss (dB)')
axes[0, 2].set_title('LoS vs NLoS Path Loss')

# Spatial path loss map
scatter = axes[1, 0].scatter(dataset['rx_x'], dataset['rx_y'], 
                            c=dataset['path_loss_db'], cmap='jet', s=20)
axes[1, 0].set_xlabel('X coordinate (m)')
axes[1, 0].set_ylabel('Y coordinate (m)')
axes[1, 0].set_title('Spatial Path Loss Map')
plt.colorbar(scatter, ax=axes[1, 0], label='Path Loss (dB)')

# Number of paths distribution
axes[1, 1].hist(dataset['num_paths'], bins=30, edgecolor='black')
axes[1, 1].set_xlabel('Number of Paths')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Multipath Distribution')

# Delay spread vs path loss
axes[1, 2].scatter(dataset['delay_spread'], dataset['path_loss_db'], alpha=0.5)
axes[1, 2].set_xlabel('Delay Spread (s)')
axes[1, 2].set_ylabel('Path Loss (dB)')
axes[1, 2].set_title('Delay Spread vs Path Loss')

plt.tight_layout()
plt.show()

# Correlation matrix
numeric_cols = dataset.select_dtypes(include=[np.number]).columns
correlation_matrix = dataset[numeric_cols].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

In [ ]:
# Save dataset
dataset.to_csv('path_loss_dataset.csv', index=False)
print("Dataset saved to 'path_loss_dataset.csv'")

## Step 7: Build Path Loss Prediction Models

**Model Selection Strategy:**
1. **Linear Regression**: Baseline, assumes linear relationship PL = β₀ + Σ βᵢ xᵢ
2. **Random Forest**: Ensemble of decision trees, captures non-linear interactions
3. **Neural Network**: Universal function approximator, learns complex mappings

**Bridge to Wireless:**
- Linear model → similar to log-distance path loss model: PL = PL₀ + 10n log(d)
- Random Forest → captures environment-specific effects (urban canyons, reflections)
- Neural Network → can model complex **shadowing** and **multipath fading**

**Evaluation Metrics:**
- **RMSE** (Root Mean Square Error): average prediction error in dB
- **R²** (Coefficient of Determination): proportion of variance explained
- **MAE** (Mean Absolute Error): average absolute prediction error

In [ ]:
# Prepare data for ML
# Select features (exclude target and identifiers)
feature_cols = ['rx_x', 'rx_y', 'rx_z', 'distance_3d', 'los_indicator',
               'num_paths', 'mean_delay', 'delay_spread', 'carrier_freq_ghz', 'tx_array_size']

X = dataset[feature_cols].values
y = dataset['path_loss_db'].values

# Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling (important for NN and linear models)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")
print(f"Number of features: {X_train.shape[1]}")

### Model 1: Linear Regression

**Theory:**
- Assumes: PL = β₀ + β₁x₁ + β₂x₂ + ... + βₙxₙ + ε
- Closed-form solution: β = (X^T X)^(-1) X^T y
- Fast training, interpretable coefficients

**Wireless Context:**
- Similar to empirical path loss models (log-distance, COST-231)
- Coefficients show linear contribution of each feature to path loss

In [ ]:
# Train Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# Predictions
y_pred_lr_train = lr_model.predict(X_train_scaled)
y_pred_lr_test = lr_model.predict(X_test_scaled)

# Evaluation
rmse_lr_train = np.sqrt(mean_squared_error(y_train, y_pred_lr_train))
rmse_lr_test = np.sqrt(mean_squared_error(y_test, y_pred_lr_test))
r2_lr_train = r2_score(y_train, y_pred_lr_train)
r2_lr_test = r2_score(y_test, y_pred_lr_test)
mae_lr_test = mean_absolute_error(y_test, y_pred_lr_test)

print("\n=== Linear Regression Results ===")
print(f"Train RMSE: {rmse_lr_train:.2f} dB")
print(f"Test RMSE: {rmse_lr_test:.2f} dB")
print(f"Train R²: {r2_lr_train:.4f}")
print(f"Test R²: {r2_lr_test:.4f}")
print(f"Test MAE: {mae_lr_test:.2f} dB")

# Print coefficients (feature importance)
print("\nFeature Coefficients:")
for feat, coef in zip(feature_cols, lr_model.coef_):
    print(f"  {feat}: {coef:.4f}")

# Visualization
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.scatter(y_test, y_pred_lr_test, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('True Path Loss (dB)')
plt.ylabel('Predicted Path Loss (dB)')
plt.title(f'Linear Regression: R²={r2_lr_test:.3f}')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
residuals = y_test - y_pred_lr_test
plt.hist(residuals, bins=30, edgecolor='black')
plt.xlabel('Residual (dB)')
plt.ylabel('Frequency')
plt.title(f'Residual Distribution (RMSE={rmse_lr_test:.2f} dB)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Model 2: Random Forest Regression

**Theory:**
- Ensemble of decision trees: f(x) = (1/M) Σ Tₘ(x)
- Each tree splits data based on feature thresholds
- Bootstrapping + feature randomness → reduces overfitting

**Advantages for Wireless:**
- Captures non-linear effects (e.g., shadowing thresholds)
- Handles mixed LoS/NLoS conditions naturally
- Provides feature importance scores

**Reference:** Breiman, L. (2001). "Random Forests." Machine Learning, 45(1), 5-32.

In [ ]:
# Train Random Forest
rf_model = RandomForestRegressor(n_estimators=100,  # number of trees
                                max_depth=20,
                                min_samples_split=5,
                                min_samples_leaf=2,
                                random_state=42,
                                n_jobs=-1)  # use all CPU cores

rf_model.fit(X_train, y_train)  # RF doesn't require scaling

# Predictions
y_pred_rf_train = rf_model.predict(X_train)
y_pred_rf_test = rf_model.predict(X_test)

# Evaluation
rmse_rf_train = np.sqrt(mean_squared_error(y_train, y_pred_rf_train))
rmse_rf_test = np.sqrt(mean_squared_error(y_test, y_pred_rf_test))
r2_rf_train = r2_score(y_train, y_pred_rf_train)
r2_rf_test = r2_score(y_test, y_pred_rf_test)
mae_rf_test = mean_absolute_error(y_test, y_pred_rf_test)

print("\n=== Random Forest Results ===")
print(f"Train RMSE: {rmse_rf_train:.2f} dB")
print(f"Test RMSE: {rmse_rf_test:.2f} dB")
print(f"Train R²: {r2_rf_train:.4f}")
print(f"Test R²: {r2_rf_test:.4f}")
print(f"Test MAE: {mae_rf_test:.2f} dB")

# Feature importance
feature_importance = rf_model.feature_importances_
print("\nFeature Importance:")
for feat, imp in sorted(zip(feature_cols, feature_importance), key=lambda x: x[1], reverse=True):
    print(f"  {feat}: {imp:.4f}")

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Predicted vs True
axes[0].scatter(y_test, y_pred_rf_test, alpha=0.5)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0].set_xlabel('True Path Loss (dB)')
axes[0].set_ylabel('Predicted Path Loss (dB)')
axes[0].set_title(f'Random Forest: R²={r2_rf_test:.3f}')
axes[0].grid(True, alpha=0.3)

# Residuals
residuals_rf = y_test - y_pred_rf_test
axes[1].hist(residuals_rf, bins=30, edgecolor='black')
axes[1].set_xlabel('Residual (dB)')
axes[1].set_ylabel('Frequency')
axes[1].set_title(f'Residual Distribution (RMSE={rmse_rf_test:.2f} dB)')
axes[1].grid(True, alpha=0.3)

# Feature importance bar plot
sorted_idx = np.argsort(feature_importance)[::-1]
axes[2].barh(range(len(feature_importance)), feature_importance[sorted_idx])
axes[2].set_yticks(range(len(feature_importance)))
axes[2].set_yticklabels([feature_cols[i] for i in sorted_idx])
axes[2].set_xlabel('Importance')
axes[2].set_title('Feature Importance')
axes[2].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

### Model 3: Neural Network (Deep Learning)

**Architecture:**
- Multi-layer perceptron (MLP): input → hidden layers → output
- Activation functions (ReLU): introduce non-linearity
- Backpropagation: gradient descent to minimize MSE loss

**Bridge to Communications:**
- **Universal approximation**: can model any continuous function
- Learns hierarchical features (low-level → high-level propagation effects)
- Similar to **deep learning for channel estimation** in massive MIMO

**References:**
- Hornik et al. (1989). "Multilayer feedforward networks are universal approximators."
- Goodfellow et al. (2016). "Deep Learning." MIT Press.

In [ ]:
# Build Neural Network
def build_nn_model(input_dim, hidden_layers=[128, 64, 32]):
    """
    Build a feedforward neural network for regression
    
    Parameters:
    -----------
    input_dim : int
        Number of input features
    hidden_layers : list
        Neurons in each hidden layer
    
    Returns:
    --------
    model : Keras model
    """
    model = models.Sequential()
    
    # Input layer
    model.add(layers.Input(shape=(input_dim,)))
    
    # Hidden layers
    for units in hidden_layers:
        model.add(layers.Dense(units, activation='relu'))
        model.add(layers.Dropout(0.2))  # Regularization
    
    # Output layer (single neuron for regression)
    model.add(layers.Dense(1, activation='linear'))
    
    # Compile
    model.compile(optimizer='adam',
                 loss='mse',
                 metrics=['mae'])
    
    return model

# Create model
nn_model = build_nn_model(input_dim=X_train_scaled.shape[1],
                         hidden_layers=[128, 64, 32])

print("Neural Network Architecture:")
nn_model.summary()

# Train with early stopping
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',
                                          patience=20,
                                          restore_best_weights=True)

history = nn_model.fit(X_train_scaled, y_train,
                      validation_split=0.2,
                      epochs=200,
                      batch_size=32,
                      callbacks=[early_stop],
                      verbose=1)

# Predictions
y_pred_nn_train = nn_model.predict(X_train_scaled).flatten()
y_pred_nn_test = nn_model.predict(X_test_scaled).flatten()

# Evaluation
rmse_nn_train = np.sqrt(mean_squared_error(y_train, y_pred_nn_train))
rmse_nn_test = np.sqrt(mean_squared_error(y_test, y_pred_nn_test))
r2_nn_train = r2_score(y_train, y_pred_nn_train)
r2_nn_test = r2_score(y_test, y_pred_nn_test)
mae_nn_test = mean_absolute_error(y_test, y_pred_nn_test)

print("\n=== Neural Network Results ===")
print(f"Train RMSE: {rmse_nn_train:.2f} dB")
print(f"Test RMSE: {rmse_nn_test:.2f} dB")
print(f"Train R²: {r2_nn_train:.4f}")
print(f"Test R²: {r2_nn_test:.4f}")
print(f"Test MAE: {mae_nn_test:.2f} dB")

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Training history
axes[0].plot(history.history['loss'], label='Training Loss')
axes[0].plot(history.history['val_loss'], label='Validation Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('MSE Loss')
axes[0].set_title('Training History')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Predicted vs True
axes[1].scatter(y_test, y_pred_nn_test, alpha=0.5)
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1].set_xlabel('True Path Loss (dB)')
axes[1].set_ylabel('Predicted Path Loss (dB)')
axes[1].set_title(f'Neural Network: R²={r2_nn_test:.3f}')
axes[1].grid(True, alpha=0.3)

# Residuals
residuals_nn = y_test - y_pred_nn_test
axes[2].hist(residuals_nn, bins=30, edgecolor='black')
axes[2].set_xlabel('Residual (dB)')
axes[2].set_ylabel('Frequency')
axes[2].set_title(f'Residual Distribution (RMSE={rmse_nn_test:.2f} dB)')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Model Comparison

In [ ]:
# Create comparison table
results_summary = pd.DataFrame({
    'Model': ['Linear Regression', 'Random Forest', 'Neural Network'],
    'Train RMSE (dB)': [rmse_lr_train, rmse_rf_train, rmse_nn_train],
    'Test RMSE (dB)': [rmse_lr_test, rmse_rf_test, rmse_nn_test],
    'Train R²': [r2_lr_train, r2_rf_train, r2_nn_train],
    'Test R²': [r2_lr_test, r2_rf_test, r2_nn_test],
    'Test MAE (dB)': [mae_lr_test, mae_rf_test, mae_nn_test]
})

print("\n=== Model Comparison ===")
print(results_summary.to_string(index=False))

# Bar plot comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

models = results_summary['Model']
x_pos = np.arange(len(models))

axes[0].bar(x_pos, results_summary['Test RMSE (dB)'])
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels(models, rotation=15)
axes[0].set_ylabel('RMSE (dB)')
axes[0].set_title('Test RMSE Comparison (Lower is Better)')
axes[0].grid(True, alpha=0.3, axis='y')

axes[1].bar(x_pos, results_summary['Test R²'])
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(models, rotation=15)
axes[1].set_ylabel('R² Score')
axes[1].set_title('Test R² Comparison (Higher is Better)')
axes[1].set_ylim([0, 1])
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## Step 8: Feature Importance Analysis

**Goal:** Identify which parameters most strongly influence path loss

**Methods:**
1. **Linear Regression**: Coefficient magnitudes (after standardization)
2. **Random Forest**: Built-in feature importance (Gini impurity reduction)
3. **Permutation Importance**: Model-agnostic method

**Physical Interpretation:**
- Distance → fundamental spreading loss (inverse square law)
- LoS/NLoS → dominant propagation condition
- Delay spread → multipath richness indicator
- Position (x,y) → spatial variations (shadowing)

In [ ]:
# 1. Linear Regression coefficients (standardized)
lr_importance = np.abs(lr_model.coef_)
lr_importance_df = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': lr_importance
}).sort_values('Importance', ascending=False)

print("=== Linear Regression Feature Importance ===")
print(lr_importance_df.to_string(index=False))

# 2. Random Forest feature importance
rf_importance_df = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\n=== Random Forest Feature Importance ===")
print(rf_importance_df.to_string(index=False))

# 3. Permutation importance for Neural Network
# (This may take a few minutes)
perm_importance = permutation_importance(nn_model, X_test_scaled, y_test,
                                        n_repeats=10,
                                        random_state=42,
                                        n_jobs=-1)

nn_importance_df = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': perm_importance.importances_mean
}).sort_values('Importance', ascending=False)

print("\n=== Neural Network Permutation Importance ===")
print(nn_importance_df.to_string(index=False))

# Visualization: side-by-side comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for ax, df, title in zip(axes, 
                         [lr_importance_df, rf_importance_df, nn_importance_df],
                         ['Linear Regression', 'Random Forest', 'Neural Network']):
    ax.barh(df['Feature'], df['Importance'])
    ax.set_xlabel('Importance')
    ax.set_title(title)
    ax.invert_yaxis()
    ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

# Physical interpretation
print("\n=== Physical Interpretation ===")
print("Distance: Expected to be important due to free-space path loss ~ d^2")
print("LoS indicator: Differentiates direct vs. obstructed paths")
print("Position (x,y): Captures spatial shadowing and environment structure")
print("Delay spread: Indicates multipath richness (more paths → more loss variation)")
print("Number of paths: Higher multipath can increase or decrease total power depending on phase")

## Step 9: Reduced-Feature Experiment

**Motivation:**
- In practice, receivers may not observe all parameters (e.g., exact path count)
- Test model performance with only **observable features**

**Observable Features:**
- RX position (known from GPS)
- LoS/NLoS (can be estimated from measurements)
- AoA (measurable with antenna arrays)
- Delay (from timing advance)
- System parameters (frequency, bandwidth, antenna gain)

**Non-Observable:**
- Exact path lengths (requires full ray tracing)
- Number of reflections/diffractions (not directly measurable)

**ML Context:**
- This tests **feature selection** and model robustness
- Analogous to **domain adaptation**: train with all features, deploy with subset

In [ ]:
# Define observable features
observable_features = ['rx_x', 'rx_y', 'rx_z', 'los_indicator', 
                      'mean_delay', 'carrier_freq_ghz', 'tx_array_size']

# Note: We remove 'distance_3d', 'num_paths', 'delay_spread' as they may not be directly observable

print("Observable features:", observable_features)
print(f"Feature reduction: {len(feature_cols)} → {len(observable_features)}")

# Prepare reduced dataset
X_obs = dataset[observable_features].values
X_train_obs, X_test_obs, y_train_obs, y_test_obs = train_test_split(
    X_obs, y, test_size=0.2, random_state=42
)

# Scale
scaler_obs = StandardScaler()
X_train_obs_scaled = scaler_obs.fit_transform(X_train_obs)
X_test_obs_scaled = scaler_obs.transform(X_test_obs)

# Train models with reduced features
print("\nTraining models with observable features only...\n")

# 1. Linear Regression
lr_obs = LinearRegression()
lr_obs.fit(X_train_obs_scaled, y_train_obs)
y_pred_lr_obs = lr_obs.predict(X_test_obs_scaled)
rmse_lr_obs = np.sqrt(mean_squared_error(y_test_obs, y_pred_lr_obs))
r2_lr_obs = r2_score(y_test_obs, y_pred_lr_obs)

print("Linear Regression (Observable):")
print(f"  Test RMSE: {rmse_lr_obs:.2f} dB")
print(f"  Test R²: {r2_lr_obs:.4f}")

# 2. Random Forest
rf_obs = RandomForestRegressor(n_estimators=100, max_depth=20, random_state=42, n_jobs=-1)
rf_obs.fit(X_train_obs, y_train_obs)
y_pred_rf_obs = rf_obs.predict(X_test_obs)
rmse_rf_obs = np.sqrt(mean_squared_error(y_test_obs, y_pred_rf_obs))
r2_rf_obs = r2_score(y_test_obs, y_pred_rf_obs)

print("\nRandom Forest (Observable):")
print(f"  Test RMSE: {rmse_rf_obs:.2f} dB")
print(f"  Test R²: {r2_rf_obs:.4f}")

# 3. Neural Network
nn_obs = build_nn_model(input_dim=X_train_obs_scaled.shape[1])
history_obs = nn_obs.fit(X_train_obs_scaled, y_train_obs,
                        validation_split=0.2,
                        epochs=200,
                        batch_size=32,
                        callbacks=[early_stop],
                        verbose=0)
y_pred_nn_obs = nn_obs.predict(X_test_obs_scaled).flatten()
rmse_nn_obs = np.sqrt(mean_squared_error(y_test_obs, y_pred_nn_obs))
r2_nn_obs = r2_score(y_test_obs, y_pred_nn_obs)

print("\nNeural Network (Observable):")
print(f"  Test RMSE: {rmse_nn_obs:.2f} dB")
print(f"  Test R²: {r2_nn_obs:.4f}")

# Comparison: Full features vs Observable features
comparison_df = pd.DataFrame({
    'Model': ['Linear Regression', 'Random Forest', 'Neural Network'],
    'Full Features RMSE': [rmse_lr_test, rmse_rf_test, rmse_nn_test],
    'Observable RMSE': [rmse_lr_obs, rmse_rf_obs, rmse_nn_obs],
    'RMSE Degradation': [
        rmse_lr_obs - rmse_lr_test,
        rmse_rf_obs - rmse_rf_test,
        rmse_nn_obs - rmse_nn_test
    ],
    'Full Features R²': [r2_lr_test, r2_rf_test, r2_nn_test],
    'Observable R²': [r2_lr_obs, r2_rf_obs, r2_nn_obs]
})

print("\n=== Performance Comparison: Full vs Observable Features ===")
print(comparison_df.to_string(index=False))

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

models = comparison_df['Model']
x = np.arange(len(models))
width = 0.35

axes[0].bar(x - width/2, comparison_df['Full Features RMSE'], width, label='Full Features')
axes[0].bar(x + width/2, comparison_df['Observable RMSE'], width, label='Observable Only')
axes[0].set_xlabel('Model')
axes[0].set_ylabel('Test RMSE (dB)')
axes[0].set_title('RMSE Comparison')
axes[0].set_xticks(x)
axes[0].set_xticklabels(models, rotation=15)
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

axes[1].bar(x - width/2, comparison_df['Full Features R²'], width, label='Full Features')
axes[1].bar(x + width/2, comparison_df['Observable R²'], width, label='Observable Only')
axes[1].set_xlabel('Model')
axes[1].set_ylabel('Test R²')
axes[1].set_title('R² Comparison')
axes[1].set_xticks(x)
axes[1].set_xticklabels(models, rotation=15)
axes[1].legend()
axes[1].set_ylim([0, 1])
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n=== Analysis ===")
print(f"Average RMSE increase: {comparison_df['RMSE Degradation'].mean():.2f} dB")
print("Interpretation: The degradation shows how much information is lost")
print("when using only receiver-observable parameters.")

## Discussion: Generalization

**Key Questions:**
1. How would this model generalize to another frequency?
2. How would it generalize to a different environment?

**Frequency Generalization:**
- Path loss scales with frequency: PL ∝ 20 log(f)
- If frequency is an input feature → model can interpolate
- **Challenge**: Propagation physics change (e.g., penetration loss)
- **Solution**: **Transfer learning** - fine-tune on small dataset at new frequency

**Environment Generalization:**
- Different scenes have different multipath statistics
- **Domain shift** problem in ML
- **Approaches:**
  - Train on multiple environments (increase diversity)
  - Use **domain adaptation** techniques
  - Include environment type as categorical feature

**Practical Deployment:**
- **Online learning**: update model as new measurements arrive
- **Hybrid approach**: combine physics-based model + ML correction
- Reference: Zappone et al. (2019). "Wireless Networks Design in the Era of Deep Learning"

In [ ]:
# Simulation: Test generalization by training on subset of space
# and testing on another region

# Split dataset spatially (e.g., train on x < 0, test on x > 0)
train_mask = dataset['rx_x'] < 0
test_mask = dataset['rx_x'] >= 0

if train_mask.sum() > 0 and test_mask.sum() > 0:
    X_spatial_train = dataset.loc[train_mask, feature_cols].values
    y_spatial_train = dataset.loc[train_mask, 'path_loss_db'].values
    X_spatial_test = dataset.loc[test_mask, feature_cols].values
    y_spatial_test = dataset.loc[test_mask, 'path_loss_db'].values
    
    # Scale
    scaler_spatial = StandardScaler()
    X_spatial_train_scaled = scaler_spatial.fit_transform(X_spatial_train)
    X_spatial_test_scaled = scaler_spatial.transform(X_spatial_test)
    
    # Train RF on one region
    rf_spatial = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    rf_spatial.fit(X_spatial_train, y_spatial_train)
    y_pred_spatial = rf_spatial.predict(X_spatial_test)
    
    rmse_spatial = np.sqrt(mean_squared_error(y_spatial_test, y_pred_spatial))
    r2_spatial = r2_score(y_spatial_test, y_pred_spatial)
    
    print("\n=== Spatial Generalization Test ===")
    print(f"Training region: x < 0 ({train_mask.sum()} samples)")
    print(f"Test region: x >= 0 ({test_mask.sum()} samples)")
    print(f"Test RMSE: {rmse_spatial:.2f} dB")
    print(f"Test R²: {r2_spatial:.4f}")
    print("\nInterpretation: This tests if the model can predict path loss")
    print("in a spatial region it hasn't seen during training.")
else:
    print("Spatial split not possible with current dataset.")

## Summary and Next Steps

**What we accomplished:**
1. ✅ Generated ray-traced propagation data using Sionna
2. ✅ Extracted physical features from channel impulse responses
3. ✅ Built and compared multiple ML models (LR, RF, NN)
4. ✅ Analyzed feature importance and physical interpretability
5. ✅ Tested reduced-feature (observable only) performance

**Key Findings:**
- Random Forest and Neural Network outperform Linear Regression
- Distance and LoS/NLoS are most important features
- Observable features alone can achieve reasonable accuracy

**Extensions for Assignment:**
- [ ] Compare M = 1, 4, 8 antenna array sizes
- [ ] Try different scenes (indoor vs outdoor)
- [ ] Experiment with hyperparameter tuning
- [ ] Add AoA/AoD features if available
- [ ] Create spatial heatmap of prediction errors

**Literature for Report:**
- Sionna documentation: https://nvlabs.github.io/sionna/
- Ray tracing for wireless: Degli-Esposti, M. (2007). "Ray tracing propagation modeling"
- ML for channel prediction: Jiang et al. (2022). "Deep learning for wireless channel modeling"